# Dask h2o benchmarks

In [6]:
import pathlib

import dask.dataframe as dd
import deltadask
import pyarrow.dataset as ds

import pandas as pd

In [2]:
pd.read_csv("~/data/G1_1e7_1e2_0_0.csv")

,id1,id2,id3,id4,id5,id6,v1,v2,v3
0,id016,id016,id0000042202,15,24,5971,5,11,37.211254
1,id039,id045,id0000029558,40,49,39457,5,4,48.951141
2,id047,id023,id0000071286,68,20,74463,2,14,60.469241
3,id043,id057,id0000015141,32,43,63743,1,15,7.692145
4,id054,id040,id0000011083,9,25,16920,2,9,22.863525
...,...,...,...,...,...,...,...,...,...
9999995,id003,id044,id0000019084,86,65,81507,4,8,30.926485
9999996,id096,id068,id0000020362,4,85,95388,3,11,53.268709
9999997,id071,id006,id0000046057,52,57,55223,1,13,90.368393
9999998,id057,id099,id0000060922,47,36,90944,3,8,34.878200


In [3]:
%%time
(
    dd.read_csv("~/data/G1_1e7_1e2_0_0.csv", usecols=["id1", "id2", "v1"])
    .query("id1 == 'id016'")
    .groupby("id2")
    .agg({"v1": "sum"})
    .compute()
)

CPU times: user 2.03 s, sys: 477 ms, total: 2.51 s
Wall time: 1.05 s


,v1
id2,
id016,3072
id061,2848
id057,2984
id017,2978
id018,3039
...,...
id011,3031
id066,2943
id076,2925


## Query 1e8 CSV with Dask

In [4]:
%%time
(
    dd.read_csv("~/data/G1_1e8_1e2_0_0.csv", usecols=["id1", "id2", "v1"])
    .query("id1 == 'id016'")
    .groupby("id2")
    .agg({"v1": "sum"})
    .compute()
)

CPU times: user 20.5 s, sys: 3.35 s, total: 23.9 s
Wall time: 9.22 s


,v1
id2,
id046,29781
id041,29440
id083,29036
id064,30300
id084,29807
...,...
id067,30424
id013,30238
id033,29769


## Query 1e8 Delta Lake with Dask

In [8]:
%%time

ddf = deltadask.read_delta(
    f"{pathlib.Path.home()}/data/delta/G1_1e8_1e2_0_0",
    columns=["id1", "id2", "v1"],
    filters=ds.field("id1") == "id016",
)
(ddf.query("id1 == 'id016'").groupby("id2").agg({"v1": "sum"}).compute())

CPU times: user 4.26 s, sys: 689 ms, total: 4.95 s
Wall time: 4.95 s


,v1
id2,
id051,29322
id046,29781
id090,30009
id017,29943
id001,29918
...,...
id065,30529
id040,30225
id078,29613
